# Option Pricing using Monte Carlo Methods

We want our code to run quickly. An example is Monte Carlo. We explore ways to make the code as fast as possible.

## Analytical Formula

We import the maths functions from the built-in maths library. N(x) needs to come from scipy.

In [4]:
from math import log, exp, sqrt
from scipy.stats import norm

def priceCallOptionAnalytical(S0,K,T,r,q,sigma):
    d1 = (log(S0/K) + (r - q + 0.5*sigma*sigma)*T) /sigma*sqrt(T)
    d2 = (log(S0/K) + (r - q - 0.5*sigma*sigma)*T) /sigma*sqrt(T)    
    value = S0 * exp(-q*T) * norm.cdf(d1,0.0,1.0) - K * exp(-r*T) * norm.cdf(d2,0.0,1.0)
    return value

In [5]:
S0 = 100.0
K = 95.0
T = 1.0
r = 0.08
q = 0.03
sigma = 0.23

In [6]:
priceCallOptionAnalytical(S0,K,T,r,q,sigma)

13.933933539010809

In [7]:
%timeit priceCallOptionAnalytical(S0,K,T,r,q,sigma)

267 µs ± 14.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Monte Carlo Valuation in Basic Python

In [8]:
numPaths = 100000

In [9]:
# Pure python code valuing a call option
import random

def priceCallOptionMC(S0,K,T,r,q,sigma,numPaths):
    payOff = 0.0
    for i in range(0,numPaths):
        z = random.gauss(0.0,1.0)
        S = S0 * exp((r-q-sigma*sigma/2.0) * T + sigma * sqrt(T) * z)        
        payOff += max(0,S-K)
    value = payOff * exp(-r*T) / numPaths
    return value

In [10]:
priceCallOptionMC(S0,K,T,r,q,sigma,numPaths)

13.973214614380598

In [11]:
%timeit priceCallOptionMC(S0,K,T,r,q,sigma,numPaths)

234 ms ± 8.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Monte Carlo Valuation in Numpy
When the function can be vectorised then Numpy can speed things up a lot

In [9]:
import numpy as np

def priceCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths):
    z = np.random.normal(size=numPaths,loc=0.0,scale=1.0) 
    S = S0 * np.exp((r-q-sigma*sigma/2.0) * T + sigma * sqrt(T) * z)
    payoff = np.maximum(S-K,0)
    value = np.sum(payoff)/numPaths * np.exp(-r*T)     
    return value

In [10]:
priceCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths)

13.928268114817378

In [11]:
%timeit priceCallOptionMC_Numpy(S0,K,T,r,q,sigma,numPaths)

6.14 ms ± 138 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Monte Carlo Valuation in Numba
Use the JIT compiler

In [12]:
from numba import njit

@njit 
def priceCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths):
    payOff = 0.0
    for i in range(0,numPaths):
        z = random.gauss(0.0,1.0)
        S = S0 * exp((r-q-sigma*sigma/2.0) * T + sigma * sqrt(T) * z)        
        payOff += max(0,S-K)
    value = payOff * exp(-r*T) / numPaths
    return value

In [13]:
priceCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths)

13.940360623272499

In [14]:
%timeit priceCallOptionMC_Numba(S0,K,T,r,q,sigma,numPaths)

4.61 ms ± 291 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
